In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from collections import Counter
import torch
from sklearn.model_selection import train_test_split
from networkx.readwrite.json_graph import adjacency
from datetime import datetime
from tool import evaluate
from tool import preprocess
from tool import customdataset
import pandas as pd
from collections import Counter
import torch
from sklearn.model_selection import train_test_split
from networkx.readwrite.json_graph import adjacency
!pip install faiss-cpu
import faiss


In [293]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# dataset_pd = pd.read_csv('D:\\VideoRecSystem\\MicroLens\\DataSet\\MicroLens-50k_pairs.csv')
path = 'MicroLens-50k_pairs.csv'
user = 'user'
item = 'item'
user_id = 'user_id'
item_id = 'item_id'
timestamp = 'timestamp'
save_dir = './embeddings'
top_k = 10
num_workers = 10
k_neg = 10
# path = pd.read_csv('MicroLens-50k_pairs.csv')
# dataset_pd = pd.read_csv('MicroLens-50k_pairs.csv')

In [295]:
dataset_pd,num_users,num_items = preprocess.openAndSort(path,user_id=user,item_id=item,timestamp='timestamp')

,user,item,timestamp
0,36121,9580,1583378629552
1,26572,9580,1583436719018
2,37550,9580,1584412681021
3,14601,9580,1584848802432
4,15061,9580,1585388171106
5,6364,9580,1585390736041
6,3542,9580,1585404918503
7,21038,9580,1590144594477
8,12538,14631,1634867362929
9,47592,14631,1634872254913


In [296]:

train_df, test_df = preprocess.split(dataset_pd,user, item, timestamp)
print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

<bound method DataFrame.count of          user   item      timestamp
0       36121   9580  1583378629552
1       26572   9580  1583436719018
2       37550   9580  1584412681021
3       14601   9580  1584848802432
4       15061   9580  1585388171106
...       ...    ...            ...
359703  48702   1363  1662984066647
359704  27203   7291  1662984082974
359705  29261  19649  1662984103874
359706  28341  19188  1662984123833
359707  38967   7254  1662984132429

[359708 rows x 3 columns]>

In [297]:
# maintain a map from new id to old id, new id for constructing matrix
user2id = {u: i for i, u in enumerate(dataset_pd[user].unique())}
item2id = {i: j for j, i in enumerate(dataset_pd[item].unique())}

# apply to train_df and test_df
train_df[user_id] = train_df[user].map(user2id)
train_df[item_id] = train_df[item].map(item2id)
test_df[user_id] = test_df[user].map(user2id)
test_df[item_id] = test_df[item].map(item2id)

<bound method DataFrame.count of          user   item      timestamp
0       36121   9580  1583378629552
1       26572   9580  1583436719018
2       37550   9580  1584412681021
3       14601   9580  1584848802432
4       15061   9580  1585388171106
...       ...    ...            ...
359703  48702   1363  1662984066647
359704  27203   7291  1662984082974
359705  29261  19649  1662984103874
359706  28341  19188  1662984123833
359707  38967   7254  1662984132429

[352649 rows x 3 columns]>

In [303]:
import numpy as np
import scipy.sparse as sp
import torch

def build_adj_matrix(df, num_users, num_items ,user_id, item_id):
    rows = df[user_id].values
    cols = df[item_id].values
    data = np.ones(len(df))
    # set interaction of user-item as 1, other as 0
    R = sp.coo_matrix((data, (rows, cols)), shape=(num_users, num_items))

    # construct symetric matrix A
    upper = sp.hstack([sp.csr_matrix((num_users, num_users)), R])
    lower = sp.hstack([R.T, sp.csr_matrix((num_items, num_items))])
    A = sp.vstack([upper, lower])

    # normalization A → Ĥ = D^{-1/2} A D^{-1/2}
    rowsum = np.array(A.sum(1)).flatten()
    d_inv_sqrt = np.power(rowsum, -0.5)
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    D_inv_sqrt = sp.diags(d_inv_sqrt)
    A_norm = D_inv_sqrt @ A @ D_inv_sqrt

    # transform to torch.sparse
    A_norm = A_norm.tocoo()
    indices = torch.LongTensor([A_norm.row, A_norm.col])
    values = torch.FloatTensor(A_norm.data)
    return torch.sparse_coo_tensor(indices, values, A_norm.shape)


In [304]:
adj_torch = build_adj_matrix(train_df, num_users, num_items ,user_id, item_id)

C:\Users\Liuds\AppData\Local\Temp\ipykernel_44728\4111138054.py:19: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5)


In [305]:
# Minimal LightGCN implementation in PyTorch

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random

class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, n_layers, adjacency):
        super(LightGCN, self).__init__()
        self.user_emb = None
        self.item_emb = None
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers
        self.adjacency = adjacency  # torch.sparse format

        self.embedding_user = nn.Embedding(num_users, embedding_dim)
        self.embedding_item = nn.Embedding(num_items, embedding_dim)

        nn.init.xavier_uniform_(self.embedding_user.weight)
        nn.init.xavier_uniform_(self.embedding_item.weight)


    def forward(self):
        all_embeddings = torch.cat([self.embedding_user.weight, self.embedding_item.weight], dim=0)
        embeddings_list = [all_embeddings]

        for _ in range(self.n_layers):
            all_embeddings = torch.sparse.mm(self.adjacency, all_embeddings)
            embeddings_list.append(all_embeddings)

        final_embedding = torch.stack(embeddings_list, dim=1).mean(dim=1)
        user_embedding, item_embedding = torch.split(final_embedding, [self.num_users, self.num_items])

        self.user_emb = user_embedding.detach()
        self.item_emb = item_embedding.detach()

        return user_embedding, item_embedding


    def get_users_embedding(self,user_ids,l2_norm=True):
        u_vec = self.user_emb[user_ids]          # (B, emb_dim)
        if l2_norm:
            u_vec = F.normalize(u_vec, p=2, dim=1)
        return u_vec

    def get_items_embedding(self,item_ids,l2_norm=True):
        i_vec = self.item_emb[item_ids]          # (B, emb_dim)
        if l2_norm:
            i_vec = F.normalize(i_vec, p=2, dim=1)
        return i_vec

    def save_embeddings(self, num_users, num_items, device, save_dir='./embeddings'):
        import os
        import faiss
        os.makedirs(save_dir, exist_ok=True)

        self.eval()
        self.to(device)

        user_ids = torch.arange(num_users, dtype=torch.long, device=device)
        item_ids = torch.arange(num_items, dtype=torch.long, device=device)

        with torch.no_grad():
            user_embeds = self.get_users_embedding(user_ids, l2_norm=True)
            item_embeds = self.get_items_embedding(item_ids, l2_norm=True)

        user_embeds = user_embeds.cpu().numpy().astype(np.float32)
        item_embeds = item_embeds.cpu().numpy().astype(np.float32)

        # 保存向量
        np.save(f"{save_dir}/user_embeddings.npy", user_embeds)
        np.save(f"{save_dir}/item_embeddings.npy", item_embeds)

        # 构建 FAISS index（使用内积）
        dim = item_embeds.shape[1]
        index = faiss.IndexFlatIP(dim)
        index.add(item_embeds)

        faiss.write_index(index, f"{save_dir}/item_index.faiss")
        print("Saved user/item embeddings and FAISS index.")


In [306]:
# def bpr_loss(user_emb, pos_item_emb, neg_item_emb):
#     pos_scores = (user_emb * pos_item_emb).sum(dim=1)
#     neg_scores = (user_emb * neg_item_emb).sum(dim=1)
#     loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()
#     return loss

In [307]:
# # Negative sampling function
# def sample_batch(train_df, num_items, batch_size):
#     users = train_df['user_id'].unique()
#     batch_users = np.random.choice(users, size=batch_size)
#     user_pos_dict = train_df.groupby('user_id')['item_id'].apply(set).to_dict()
#
#     user_ids, pos_ids, neg_ids = [], [], []
#     for u in batch_users:
#         pos_items = list(user_pos_dict[u])
#         pos = random.choice(pos_items)
#         while True:
#             neg = random.randint(0, num_items - 1)
#             if neg not in user_pos_dict[u]:
#                 break
#         user_ids.append(u)
#         pos_ids.append(pos)
#         neg_ids.append(neg)
#
#     return torch.LongTensor(user_ids), torch.LongTensor(pos_ids), torch.LongTensor(neg_ids)

In [308]:
import torch
import numpy as np

# --------- 采样函数保持不变 (示意) ------------
# def sample_batch(train_df, num_items, batch_size): ...

# --------- BPR 损失保持不变 -------------------
# def bpr_loss(user_vec, pos_vec, neg_vec): ...

def train_model(model,
                train_df,
                num_items,
                epochs=50,
                batch_size=1024,
                lr=1e-3,
                device=None):
    """
    训练 LightGCN (或其它 BPR 模型) 的通用函数
    ------------------------------------------------
    • train_df      : pandas DataFrame，含 user_id / item_id
    • num_items     : 物品总数
    • device        : torch.device；默认为 'cuda' (若可用) 否则 'cpu'
    • max_grad_norm : 梯度裁剪阈值；避免梯度爆炸，可选
    """
    # -------- 设备 ----------
    if device is None:
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    if hasattr(model, "adjacency"):               # adjacency 可能是稀疏张量
        model.adjacency = model.adjacency.to(device)

    # -------- 优化器 ----------
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_loader = customdataset.build_train_loader_inbatch(train_df, batch_size=batch_size,user_col=user_id, item_col=item_id)

    # -------- 训练循环 ----------
    for epoch in range(1, epochs + 1):
        model.train()
        dt_start = datetime.now()
        epoch_loss = 0.0

        for batch in train_loader:
            model()
            user_ids, pos_item_ids = batch
            user_ids = user_ids.to(device)
            pos_item_ids = pos_item_ids.to(device)
            u_vec = model.get_users_embedding(user_ids,l2_norm=False)
            i_vec = model.get_embedding(pos_item_ids,l2_norm=False)

            # 2. 得分矩阵：每个 user 对所有正 item 的打分
            logits = torch.matmul(u_vec, i_vec.T)  # shape: (B, B)

            # 3. 构造标签：每个 user 的正确 item 在对角线（即位置 i）
            labels = torch.arange(logits.size(0), device=device)  # [0, 1, ..., B-1]

            # 4. Cross Entropy Loss
            loss = F.cross_entropy(logits, labels)

            # 5. 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # 日志
        avg_loss = epoch_loss / len(train_loader)
        dt_end = datetime.now()
        dt = dt_end - dt_start

        print(f"[Epoch {epoch:02d}/{epochs}] avg InBatch Softmax Loss = {avg_loss:.4f}, time = {dt.total_seconds():.2f}s")

    return


In [312]:

def evaluate_ranking(
        test_df,              # DataFrame, 必含 user_id / item_id
        train_df,             # DataFrame, 用来构建用户→已交互物品集合
        score_fn,             # callable(users_tensor, items_tensor) → np.array
        num_items,            # 物品总数
        k=10,                 # Hit@K / NDCG@K
        num_neg=100,          # 每个正样本采多少负样本
        user_col='user_id',
        item_col='item_id',
        seed=42
    ):
    """
    不依赖具体模型，只要提供 score_fn 就能评估。
    score_fn: 接收 (user_tensor, item_tensor) 并返回同长度的 Numpy 分数向量。
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # 用户历史，用于采负样本 & 过滤
    train_user_dict = (
        train_df.groupby(user_col)[item_col].apply(set).to_dict()
    )

    hits, ndcgs = [], []

    for _, row in test_df.iterrows():
        u = int(row[user_col])
        pos_item = int(row[item_col])

        # ---------- 负采样 ----------
        neg_items = set()
        while len(neg_items) < num_neg:
            neg = random.randint(0, num_items - 1)
            if neg not in train_user_dict.get(u, set()) and neg != pos_item:
                neg_items.add(neg)

        item_candidates = list(neg_items) + [pos_item]

        # ---------- 评分 ----------
        users_t  = torch.LongTensor([u] * len(item_candidates))
        items_t  = torch.LongTensor(item_candidates)
        scores   = score_fn(users_t, items_t)        # ← 只依赖 score_fn
        rank_idx = np.argsort(scores)[::-1]          # 降序
        ranked_items = [item_candidates[i] for i in rank_idx]

        # ---------- 指标 ----------
        if pos_item in ranked_items[:k]:
            hits.append(1)
            rank_pos = ranked_items.index(pos_item)
            ndcgs.append(1 / np.log2(rank_pos + 2))
        else:
            hits.append(0)
            ndcgs.append(0)

    hit_rate = float(np.mean(hits))
    ndcg     = float(np.mean(ndcgs))
    return hit_rate, ndcg

In [313]:
# # Top-K recommendation for a user
# def recommend_top_k(model, user_id, train_df, k=10):
#     model.eval()
#     user_emb, item_emb = model.get_embedding()
#     seen_items = set(train_df[train_df['user_id'] == user_id]['item_id'])
#     all_items = torch.arange(model.num_items)
#     scores = model.predict(torch.LongTensor([user_id] * model.num_items), all_items).detach().numpy()
#
#     ranked_items = np.argsort(scores)[::-1]
#     recommended = [i for i in ranked_items if i not in seen_items][:k]
#     return recommended

In [314]:
# # Save and load model
#
# def save_model(model, path):
#     torch.save(model.state_dict(), path)
#
# def load_model(model, path):
#     model.load_state_dict(torch.load(path))
#     model.eval()

In [315]:
model = LightGCN(num_users,num_items,embedding_dim=128,n_layers=2,adjacency=adj_torch)
model.to(device)
train_model(model=model,epochs=100, train_df=train_df,num_items=num_items,batch_size=1024)

[Epoch 1/300]  BPR Loss = 0.6930
[Epoch 2/300]  BPR Loss = 0.6929
[Epoch 3/300]  BPR Loss = 0.6927
[Epoch 4/300]  BPR Loss = 0.6924
[Epoch 5/300]  BPR Loss = 0.6919
[Epoch 6/300]  BPR Loss = 0.6915
[Epoch 7/300]  BPR Loss = 0.6907
[Epoch 8/300]  BPR Loss = 0.6901
[Epoch 9/300]  BPR Loss = 0.6894
[Epoch 10/300]  BPR Loss = 0.6885
[Epoch 11/300]  BPR Loss = 0.6874
[Epoch 12/300]  BPR Loss = 0.6864
[Epoch 13/300]  BPR Loss = 0.6831
[Epoch 14/300]  BPR Loss = 0.6836
[Epoch 15/300]  BPR Loss = 0.6821
[Epoch 16/300]  BPR Loss = 0.6802
[Epoch 17/300]  BPR Loss = 0.6785
[Epoch 18/300]  BPR Loss = 0.6748
[Epoch 19/300]  BPR Loss = 0.6758
[Epoch 20/300]  BPR Loss = 0.6731
[Epoch 21/300]  BPR Loss = 0.6696
[Epoch 22/300]  BPR Loss = 0.6662
[Epoch 23/300]  BPR Loss = 0.6620
[Epoch 24/300]  BPR Loss = 0.6581
[Epoch 25/300]  BPR Loss = 0.6604
[Epoch 26/300]  BPR Loss = 0.6537
[Epoch 27/300]  BPR Loss = 0.6526
[Epoch 28/300]  BPR Loss = 0.6506
[Epoch 29/300]  BPR Loss = 0.6474
[Epoch 30/300]  BPR Los

0.13158681988716125

In [ ]:
model.save_embeddings(num_users=num_users,num_items=num_items,device=device,save_dir=save_dir)

In [ ]:
test_loader = customdataset.build_test_loader(test_df, num_items ,user_col = user_id, item_col = item_id, batch_size=1024, num_workers=num_workers)
item_pool = list(range(num_items))
faiss_index = faiss.read_index(f"{save_dir}/item_index.faiss")

In [ ]:
hr_r, ndcg_r = evaluate.evaluate_random(test_loader, item_pool ,top_k=top_k)
print(f"Random HR@{top_k} = {hr_r:.4f}, nDCG@{top_k} = {ndcg_r:.4f}")
hr_p, ndcg_p = evaluate.evaluate_popular(test_loader, train_df,top_k=top_k)
print(f"Popular HR@{top_k} = {hr_p:.4f}, nDCG@{top_k} = {ndcg_p:.4f}")
hr_m, ndcg_m = evaluate.evaluate_model(test_loader, model, faiss_index, device,top_k=top_k)
print(f"Model   HR@{top_k} = {hr_m:.4f}, nDCG@{top_k} = {ndcg_m:.4f}")


In [ ]:
# def random_score_fn(users_t, items_t):
#     # 随机给每个 items_t 一个分数；users_t 不使用，但必须接收
#     return np.random.rand(len(items_t))

In [ ]:
# def make_popularity_score_fn(train_df, item_col='item_id'):
#     item_cnt = Counter(train_df[item_col])
#     default_score = min(item_cnt.values()) - 1  # 给没出现过的物品一个更低分
#     def _score_fn(users_t, items_t):
#         return np.array([item_cnt.get(int(i), default_score) for i in items_t])
#     return _score_fn

In [ ]:
# def make_lightgcn_score_fn(model):
#     model.eval()
#     with torch.no_grad():
#         user_emb, item_emb = model.get_embedding()
#         user_emb = user_emb.to("cuda" if torch.cuda.is_available() else "cpu")
#         item_emb = item_emb.to("cuda" if torch.cuda.is_available() else "cpu")
#
#     def score_fn(users_t, items_t):
#         u_vec = user_emb[users_t]
#         i_vec = item_emb[items_t]
#         return torch.sum(u_vec * i_vec, dim=1).cpu().numpy()
#     return score_fn


In [316]:
# pop_score_fn  = make_popularity_score_fn(train_df)
# # ---------------- baseline：Popular ----------------
# hit_pop, ndcg_pop = evaluate_ranking(
#     test_df, train_df, pop_score_fn,
#     num_items=model.num_items, k=10
# )
# print(f"Popular  Hit@10={hit_pop:.4f}  NDCG@10={ndcg_pop:.4f}")
#
# # ---------------- baseline：Random -----------------
# hit_rand, ndcg_rand = evaluate_ranking(
#     test_df, train_df, random_score_fn,
#     num_items=model.num_items, k=10
# )
# print(f"Random   Hit@10={hit_rand:.4f}  NDCG@10={ndcg_rand:.4f}")
#
# # ---------------- LightGCN（或其他模型）------------
# score_fn_gcn = make_lightgcn_score_fn(model)
# hit_gcn, ndcg_gcn = evaluate_ranking(
#     test_df, train_df, score_fn_gcn,
#     num_items=model.num_items, k=10
# )
# print(f"LightGCN Hit@10={hit_gcn:.4f}  NDCG@10={ndcg_gcn:.4f}")

Popular  Hit@10=0.2166  NDCG@10=0.1081
Random   Hit@10=0.0996  NDCG@10=0.0455
LightGCN Hit@10=0.4841  NDCG@10=0.2900
